In [ ]:
!pip install 'litellm[proxy]'

In [ ]:
!pip install vllm

In [3]:
!nohup vllm serve Qwen/Qwen3-1.7B --port 4000 --gpu-memory-utilization 0.8 > vllm.log 2>&1 &

In [4]:
!curl -s http://localhost:4000/v1/models

In [32]:
!curl -s http://127.0.0.1:4000/v1/chat/completions -H "Content-Type: application/json" -d '{"model":"Qwen/Qwen3-1.7B","messages":[{"role":"user","content":"Say hello in five words."}], "max_tokens": 64}'

{"id":"chatcmpl-23ca0e0b708a4df7b5ca7bb258a49450","object":"chat.completion","created":1758599344,"model":"Qwen/Qwen3-1.7B","choices":[{"index":0,"message":{"role":"assistant","content":"<think>\nOkay, the user wants me to say hello in five words. Let me think about how to approach this. First, I need to make sure I understand the request correctly. They want a greeting that's concise, exactly five words. \n\nHmm, the most common way to say hello is \"Hello!\" but","refusal":null,"annotations":null,"audio":null,"function_call":null,"tool_calls":[],"reasoning_content":null},"logprobs":null,"finish_reason":"length","stop_reason":null,"token_ids":null}],"service_tier":null,"system_fingerprint":null,"usage":{"prompt_tokens":14,"total_tokens":78,"completion_tokens":64,"prompt_tokens_details":null},"prompt_logprobs":null,"prompt_token_ids":null,"kv_transfer_params":null}

look at log file

In [7]:
!tail -200 vllm.log

2025-09-23 04:10:35.942818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758600635.973685    2025 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758600635.982735    2025 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758600636.003064    2025 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758600636.003095    2025 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758600636.003099    2025 computation_placer.cc:177] computation placer alr

In [10]:
import litellm

response = litellm.completion(
    model="hosted_vllm/Qwen/Qwen3-1.7B",
    messages=[{"role": "user", "content": "hello"}],
    api_base="http://127.0.0.1:4000/v1",
    api_key="sk-local",  # this will work for local
    temperature=0.2,
    max_tokens=10,
)

print(response)

ModelResponse(id='chatcmpl-8de5cb53939b455696880c88e714d9ac', created=1758601047, model='hosted_vllm/Qwen/Qwen3-1.7B', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='length', index=0, message=Message(content='<think>\nOkay, the user said "hello".', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}), provider_specific_fields={'stop_reason': None, 'token_ids': None})], usage=Usage(completion_tokens=10, prompt_tokens=9, total_tokens=19, completion_tokens_details=None, prompt_tokens_details=None), service_tier=None, prompt_logprobs=None, prompt_token_ids=None, kv_transfer_params=None)


In [11]:
import json
import litellm

api_base = "http://127.0.0.1:4000/v1"
api_key = "sk-local"
model = "hosted_vllm/Qwen/Qwen3-1.7B"

tools = [
    {
        "type": "function",
        "function": {
            "name": "submit_paths",
            "description": "Submit the top P shortest paths found",
            "parameters": {
                "type": "object",
                "properties": {
                    "paths": {"type": "array", "items": {"type": "array", "items": {"type": "integer"}}},
                    "weights": {"type": "array", "items": {"type": "integer"}},
                },
                "required": ["paths", "weights"],
            },
        },
    }
]

resp = litellm.completion(
    model=model,
    api_base=api_base,
    api_key=api_key,
    messages=[{"role":"user","content":"Return two example paths using submit_paths."}],
    tools=tools,
    tool_choice={"type":"function","function":{"name":"submit_paths"}},
    temperature=0,
    max_tokens=256,
)

msg = resp.choices[0].message
tool_calls = getattr(msg, "tool_calls", None) or msg.get("tool_calls", [])
if tool_calls:
    args = json.loads(tool_calls[0]["function"]["arguments"])
    print("Tool args:", args)
else:
    print("No tool call returned:", msg)

Tool args: {'paths': [[1, 2, 3], [1, 4, 3]], 'weights': [1, 2]}
